In [1]:
!pip install tensorflow_probability
!pip install -U scikit-learn
!pip install hpfrec
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/uk2051/.local/lib/python3.8/site-packages (1.0.2)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [2]:
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import numpy.random as npr
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
#import statsmodels.api as sm
#import networkx as nx

#from tensorflow_probability import edward2 as ed
#from sklearn.datasets import load_breast_cancer
from pandas.plotting import scatter_matrix
from scipy import sparse, stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, mean_squared_error, r2_score
from sklearn.utils import shuffle

import matplotlib
matplotlib.rcParams.update({'font.sans-serif' : 'Helvetica',
                            'axes.labelsize': 10,
                            'xtick.labelsize' : 6,
                            'ytick.labelsize' : 6,
                            'axes.titlesize' : 10})
import matplotlib.pyplot as plt

#import seaborn as sns
#color_names = ["windows blue",
#               "amber",
#               "crimson",
#               "faded green",
#               "dusty purple",
#               "greyish"]
#colors = sns.xkcd_palette(color_names)
#sns.set(style="white", palette=sns.xkcd_palette(color_names), color_codes = False)


/home/uk2051/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/uk2051/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/uk2051/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/uk2051/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [11]:
#df_o = pd.read_csv("/content/drive/MyDrive/ML_Project_Datasets/train_0.txt", sep="\t", header=None,
#                 names=["UserId", "SongId", "Ratings"])
#df_o = pd.read_csv("/content/drive/MyDrive/ML_Project_Datasets/train_0.txt", sep="\t", header=None,
#                 names=["UserId", "ItemId", "Count"])
df_o = pd.read_csv("/home/uk2051/ML_Project/train_0.txt", sep="\t", header=None,
                 names=["UserId", "ItemId", "Count"])
print(df_o.shape)

(76344627, 3)


In [12]:
maxUserId = df_o["UserId"].max()
minUserId = df_o["UserId"].min()
maxSongId = df_o["ItemId"].max()
minSongId = df_o["ItemId"].min()

totalUsers = maxUserId +1
totalItems = maxSongId + 1
matrix = sparse.coo_matrix((df_o["Count"],(df_o["UserId"],df_o["ItemId"])),shape=(totalUsers,totalItems))

#Creating a matrix
#First changing the datafram last column to only contain 1
df_o['Count'] = df_o['Count'].where(df_o['Count'] == 0, 1) #Modified the original dataframe, look into it

a_matrix = sparse.coo_matrix((df_o["Count"],(df_o["UserId"],df_o["ItemId"])),shape=(totalUsers,totalItems))

#matrix = matrix.tocsr()[:10000,:5000].tocoo()
#a_matrix = a_matrix.tocsr()[:10000,:5000].tocoo()
matrix = matrix.tocsr().tocoo()
a_matrix = a_matrix.tocsr().tocoo()

In [13]:
print(matrix.shape)

(200000, 136736)


In [14]:
print(a_matrix.shape)

(200000, 136736)


In [15]:
import pandas as pd, numpy as np
from hpfrec import HPF
np.random.seed(1)


Parameters
	----------
	k : int -> Number of latent factors to use.
	a : float -> Shape parameter for the user-factor matrix.
	a_prime : float -> Shape parameter and dividend of the rate parameter for the user activity vector.
	b_prime : float -> Divisor of the rate parameter for the user activity vector.
	c : float -> Shape parameter for the item-factor matrix.
	c_prime : float -> Shape parameter and dividend of the rate parameter for the item popularity vector.
	d_prime : float -> Divisor o the rate parameter for the item popularity vector.


In [16]:
a_matrix_copy = a_matrix.copy()

def ReturnParamsFromHPF():
    ## Full function call
    recommender = HPF(
      k=10, 
      a=0.3, 
      a_prime=0.3, 
      b_prime=1.0,
      c=0.3, 
      c_prime=0.3, 
      d_prime=1.0,
      maxiter=60,
      verbose=True,
      produce_dicts=False, 
      keep_all_objs=False,
      save_folder ="/home/uk2051/ML_Project/Params")
    recommender.fit(a_matrix_copy)
    theta_users = recommender.Theta
    eta_items = np.transpose(recommender.Beta)

    
    return theta_users, eta_items
    
theta_users, beta_items = ReturnParamsFromHPF()

**********************************
Hierarchical Poisson Factorization
**********************************

Number of users: 200000
Number of items: 136736
Latent factors to use: 10

Initializing parameters...
Allocating Phi matrix...
Initializing optimization procedure...
Iteration 10 | train llk: -385939044 | train rmse: 0.9496
Iteration 20 | train llk: -341807546 | train rmse: 0.9250
Iteration 30 | train llk: -331704043 | train rmse: 0.9170
Iteration 40 | train llk: -330216810 | train rmse: 0.9162
Iteration 50 | train llk: -329622779 | train rmse: 0.9157
Iteration 60 | train llk: -329312690 | train rmse: 0.9155


Optimization finished
Final log-likelihood: -329312690
Final RMSE: 0.9155
Minutes taken (optimization part): 13.5

Saving final parameters to .csv files...
